In [54]:
import pandas as pd
import numpy as np

#corpuse featurs (to combine for cossine similairity)
#genres, overview, production_companies, tagline, cast, crew, keywords
#quantitative: vote_average, rating, user rating average...

#be sure to check the column data t


pd.set_option('display.max_colwidth', None)

moviesFull = pd.read_csv('newdata/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
ratings = pd.read_csv('newdata/ratings.csv')
keywords = pd.read_csv('newdata/keywords.csv')
credits = pd.read_csv("newdata/credits.csv")

credits = credits.drop(['crew'], axis=1)
moviesFull = moviesFull.dropna(subset = ["tagline","overview"])

In [55]:
#test format of data

print(moviesFull.iloc[0])
print(keywords.iloc[0])
print(credits.iloc[0])

genres                                                                                                                                                                                                                                                                                                                            [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]
id                                                                                                                                                                                                                                                                                                                                                                                                                             8844
overview                When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an

In [56]:
import json
import re



def condition(id):
    #accounted for:
    #keywords, crew, cast, tagline, prodution_companies,genres, overview

    #note: tagline and overfiew are droped for na values
    if id not in moviesFull["id"]:
        return False
    if(len(keywords[keywords["id"] == id]["keywords"]))!=0:
        return False
    if(len(credits[credits["id"] == id]["cast"])!=0):
        return False
    if(len(moviesFull[moviesFull["id"] == str(id)]["production_companies"])!=0):
        return False
    if(len(moviesFull[moviesFull["id"] == str(id)]["genres"])!= 0):
        return False
    
    return True



def provideData(movieData):
    movieData.append(id)
    movieData.append(rating)

    regex = r"'name': '[\sa-zA-Z]*'}";

    matches  = re.findall(regex,str(keywords[keywords["id"] == id]["keywords"]))
    movieData.append(' '.join([item[9:-2] for item in matches]))
    matches  = re.findall(regex,str(moviesFull[moviesFull["id"] == id]["genres"]))
    movieData.append(' '.join([item[9:-2] for item in matches]))

    regex = r"'name': '[\sa-zA-Z]*,";

    matches  = re.findall(regex,str(credits[credits["id"] == id]["cast"]))
    movieData.append(' '.join([item[9:-2] for item in matches]))
    matches  = re.findall(regex,str(moviesFull[moviesFull["id"] == id]["production_companies"]))
    movieData.append(' '.join([item[9:-2] for item in matches]))


    movieData.append(str(moviesFull[moviesFull["id"] == id]["tagline"]))
    movieData.append(str(moviesFull[moviesFull["id"] == id]["overview"]))


#note: ratings do not need to be filtered there are no nans
#take a thousand users
userDict = dict()


for i in range(1, 1001):

    userRatings = ratings[ratings["userId"] ==i]

    filtered = []
    #for each user there is a list of movies
    #for each movie there is a list of strings
    #the list of string should be combined for each movie and then vetcorized

    for id,rating in zip(list(userRatings["movieId"]), list(userRatings["rating"])):
        if (condition(id)):
            movieData  = []
            provideData(movieData)
            filtered.append(movieData)

    #should there be a minimum number of ratings here? 
    if(not len(filtered) == []):
        userDict[i] = filtered






In [57]:
from gensim.parsing.preprocessing import remove_stopwords

combinedFeatures = []
#list of users each with a list of movies data each with a list of data strings

for key in userDict.keys():
    movieStrings = []
    for movieData in userDict[key]:
        movieString = ""
        #avoid the first two data points id and rating
        for index in range (2,len(movieData)):
            movieString+= movieData[index]+" "
        cleaned = remove_stopwords(movieString)
        movieStrings.append(cleaned)
    combinedFeatures.append(movieStrings)



